# Лабораторная работа: Рекомендательные системы

## Теоретическая часть

### 1. Суть задачи рекомендательных систем
Рекомендательные системы – это алгоритмы, которые анализируют поведение пользователей и предлагают персонализированные рекомендации товаров, фильмов, музыки и других объектов. Основная цель – предсказать предпочтения пользователей на основе имеющихся данных о взаимодействиях.

### 2. Метод коллаборативной фильтрации
Коллаборативная фильтрация (Collaborative Filtering, CF) – это метод рекомендаций, основанный на анализе поведения пользователей. Он работает на основе предположения, что пользователи с похожими предпочтениями в прошлом будут делать схожий выбор в будущем.

Существует два основных подхода:
1. **User-based CF** – рекомендации строятся на основе сходства пользователей.
2. **Item-based CF** – рекомендации строятся на основе сходства объектов.

### 3. Латентные факторные модели (Matrix Factorization)
Коллаборативная фильтрация может быть реализована через матричное разложение. Пусть у нас есть матрица взаимодействий пользователей и объектов R, где $( R_{u,i} )$ – оценка пользователя ( u ) для объекта ( i ). Тогда разложение можно представить в виде:
$$
R \approx U \cdot V^T
$$
где:
- ( U ) – матрица эмбеддингов пользователей,
- ( V ) – матрица эмбеддингов объектов.

Предсказание рейтинга рассчитывается как:
$$
\hat{R}_{u,i} = U_u \cdot V_i^T
$$

В данной лабораторной работе предполагается использование **нейросетевого метода**, который обучает эмбеддинги пользователей и объектов с помощью полносвязных слоев. Входные данные – индексы пользователей и объектов, которые преобразуются в векторные представления, а затем подаются на вход нейросети.


## Практическая часть
В данной работе вам предлагается реализовать рекомендательную систему на основе метода коллаборативной фильтрации, используя нейросетевую модель. Вы должны:
1. Подготовить данные: загрузить свой датасет (например, рейтинг фильмов, товаров, книг и т. д.).
2. Разбить данные на тренировочный и тестовый наборы.
3. Обучить модель, используя эмбеддинги пользователей и объектов.
4. Оценить качество модели на тестовом наборе.
5. Вывести список рекомендаций для выбранного пользователя.

In [63]:
# Импорты
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Определяем устройство (используем GPU, если доступно)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [67]:
anime_df = pd.read_csv('animes.csv')
# Загружаем rating.csv 
ratings_df = pd.read_csv('ratings.csv')
ratings_df = ratings_df.drop_duplicates()

# Фильтруем только положительные рейтинги ( > 0, поскольку 0 - без рейтинга)
ratings_df = ratings_df[ratings_df['rating'] > 0]

# Преобразуем идентификаторы пользователей и аниме (начинаем с 0 для удобства в PyTorch)
user_id_map = {old: new for new, old in enumerate(ratings_df['user_id'].unique())}
anime_id_map = {old: new for new, old in enumerate(ratings_df['anime_id'].unique())}

ratings_df['user_idx'] = ratings_df['user_id'].map(user_id_map)
ratings_df['anime_idx'] = ratings_df['anime_id'].map(anime_id_map)

print(f"До обрезки:")
print(f"  пользователей: {ratings_df['user_idx'].nunique():,}")
print(f"  аниме:         {ratings_df['anime_idx'].nunique():,}")
print(f"  взаимодействий: {len(ratings_df):,}")
print(ratings_df.head())

user_activity = ratings_df['user_id'].value_counts()
N = 3000
top_users = user_activity.head(N).index

# Оставляем только их
ratings_df = ratings_df[ratings_df['user_id'].isin(top_users)].copy()

# Обязательно перестраиваем индексы после фильтрации!
ratings_df = ratings_df.reset_index(drop=True)

user_id_map = {old: new for new, old in enumerate(ratings_df['user_id'].unique())}
anime_id_map = {old: new for new, old in enumerate(ratings_df['anime_id'].unique())}

ratings_df['user_idx']  = ratings_df['user_id'].map(user_id_map)
ratings_df['anime_idx'] = ratings_df['anime_id'].map(anime_id_map)

print("После обрезки:")
print(f"  пользователей: {ratings_df['user_idx'].nunique():,}")
print(f"  аниме:         {ratings_df['anime_idx'].nunique():,}")
print(f"  взаимодействий: {len(ratings_df):,}")



До обрезки:
  пользователей: 82,519
  аниме:         11,439
  взаимодействий: 9,015,302
   user_id  anime_id  rating  user_idx  anime_idx
0        1       454       3         0          0
1        1     28761       8         0          1
2        1      6682       5         0          2
3        1      9624       6         0          3
4        1     38101       7         0          4
После обрезки:
  пользователей: 3,000
  аниме:         10,736
  взаимодействий: 1,887,252


In [ ]:
# Определяем датасет PyTorch
class RatingsDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['user_idx'].values, dtype=torch.long)
        self.items = torch.tensor(df['anime_idx'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['rating'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

In [ ]:
# Определяем нейросетевую модель для коллаборативной фильтрации
class RecommenderNN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=32):
        super(RecommenderNN, self).__init__()
        # Эмбеддинги пользователей и аниме
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        # Полносвязные слои для предсказания рейтинга
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_dim * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, user, item):
        # Получаем эмбеддинги пользователя и аниме
        user_emb = self.user_embedding(user)
        item_emb = self.item_embedding(item)

        # Объединяем эмбеддинги
        x = torch.cat([user_emb, item_emb], dim=1)

        # Пропускаем через полносвязные слои
        return self.fc_layers(x).squeeze()

In [ ]:
# Определяем количество пользователей и аниме
num_users = len(user_id_map)
num_items = len(anime_id_map)

In [ ]:
# Создаём датасеты и загрузчики данных
dataset = RatingsDataset(ratings_df)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
# Инициализация модели
model = RecommenderNN(num_users, num_items).to(device)

# Определяем функцию потерь (MSE) и оптимизатор (Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [60]:
epochs = 10

# Обучение модели
for epoch in range(epochs):
    model.train()
    total_loss = 0
    all_predictions = []
    all_ratings = []
    for users, items, ratings in train_loader:
        users, items, ratings = users.to(device), items.to(device), ratings.to(device)
        optimizer.zero_grad()
        predictions = model(users, items)
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        all_predictions.extend(predictions.cpu().detach().numpy())
        all_ratings.extend(ratings.cpu().detach().numpy())

    # Средняя ошибка предсказания на тренировочной выборке
    rmse = math.sqrt(mean_squared_error(all_ratings, all_predictions))
    mae = mean_absolute_error(all_ratings, all_predictions)

    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}, Train RMSE: {rmse:.4f}, Train MAE: {mae:.4f}')


Epoch 1, Loss: 1.536280402317179, Train RMSE: 1.2395, Train MAE: 0.9379
Epoch 2, Loss: 1.3315215431970808, Train RMSE: 1.1539, Train MAE: 0.8708
Epoch 3, Loss: 1.2784823856533754, Train RMSE: 1.1307, Train MAE: 0.8514
Epoch 4, Loss: 1.2406577561654666, Train RMSE: 1.1138, Train MAE: 0.8379
Epoch 5, Loss: 1.2136707895347634, Train RMSE: 1.1017, Train MAE: 0.8278
Epoch 6, Loss: 1.18782865750871, Train RMSE: 1.0899, Train MAE: 0.8184
Epoch 7, Loss: 1.1636571633796058, Train RMSE: 1.0787, Train MAE: 0.8098
Epoch 8, Loss: 1.1381805875331075, Train RMSE: 1.0669, Train MAE: 0.8001
Epoch 9, Loss: 1.1140658042193985, Train RMSE: 1.0555, Train MAE: 0.7915
Epoch 10, Loss: 1.090787140385272, Train RMSE: 1.0444, Train MAE: 0.7831


На каждой следующей эпохе метрики улучшаются, нет переобучения.

In [61]:
# Оценка модели на тестовом наборе
model.eval()
test_predictions = []
test_ratings = []
with torch.no_grad():
    for users, items, ratings in test_loader:
        users, items, ratings = users.to(device), items.to(device), ratings.to(device)
        predictions = model(users, items)
        test_predictions.extend(predictions.cpu().numpy())
        test_ratings.extend(ratings.cpu().numpy())

# Средняя ошибка на тестовом наборе
test_rmse = math.sqrt(mean_squared_error(test_ratings, test_predictions))
test_mae = mean_absolute_error(test_ratings, test_predictions)

print(f'\nTest RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}')


Test RMSE: 1.1293, Test MAE: 0.8505


- RMSE: 1.1293 — модель в среднем ошибается на ~1.13 балла в предсказании рейтинга. Это приемлемый результат, указывающий на то, что предсказания близки к реальным
- MAE: 0.8505 — средняя абсолютная ошибка ~0.85 балла, что подтверждает стабильность модели. MAE ниже RMSE говорит о том, что крупных выбросов ошибок мало.

In [62]:
idx_to_anime = {v: k for k, v in anime_id_map.items()}

# Выбираем случайных пользователей (оригинальные user_id)
random_users = np.random.choice(ratings_df['user_id'].unique(), size=5)

print("\nПерсонализированные рекомендации (топ-5 аниме) для случайных пользователей:")
for user_id in random_users:
    user_idx = user_id_map[user_id]
    
    # Предсказания для всех аниме для выбранного пользователя
    user_tensor = torch.tensor([user_idx] * num_items, dtype=torch.long).to(device)
    item_tensor = torch.tensor(range(num_items), dtype=torch.long).to(device)

    with torch.no_grad():
        predictions = model(user_tensor, item_tensor).cpu().numpy()

    # Выбираем топ-5 рекомендованных аниме (idx)
    top_indices = predictions.argsort()[-5:][::-1]
    top_anime_ids = [idx_to_anime[idx] for idx in top_indices]
    
    # Получаем названия
    top_titles = anime_df[anime_df['anime_id'].isin(top_anime_ids)]['title'].tolist()

    print(f"Пользователь {user_id}: Рекомендуемые аниме: {top_titles}")


Персонализированные рекомендации (топ-5 аниме) для случайных пользователей:
Пользователь 81028: Рекомендуемые аниме: ['Monster', 'Ginga Eiyuu Densetsu', 'Fullmetal Alchemist: Brotherhood', 'Gintama・ゑｽｰ', 'Violet Evergarden Movie']
Пользователь 100770: Рекомендуемые аниме: ['Clannad: After Story', "Gintama': Enchousen", 'Digimon Adventure: Last Evolution Kizuna', 'Kimetsu no Yaiba Movie: Mugen Ressha-hen']
Пользователь 36894: Рекомендуемые аниме: ['Great Teacher Onizuka', 'Ginga Eiyuu Densetsu', 'Code Geass: Hangyaku no Lelouch R2', 'Evangelion: 2.0 You Can (Not) Advance', 'Fullmetal Alchemist: Brotherhood']
Пользователь 3394: Рекомендуемые аниме: ['Hellsing Ultimate', 'Clannad: After Story', 'Fullmetal Alchemist: Brotherhood', 'Angel Beats!', 'Kimetsu no Yaiba Movie: Mugen Ressha-hen']
Пользователь 29984: Рекомендуемые аниме: ['Bishoujo Senshi Sailor Moon S', 'Yokohama Kaidashi Kikou: Quiet Country Cafe', 'Ashita no Joe', 'Gosenzo-sama Banbanzai!', 'Hyouge Mono']


Ключевые этапы:
1. Подготовка датасета
2. Подготовка и обучение модели
3. Оценка модели на тестовом наборе
4. Вычисление метрик

Для улучшения метрик можно увеличить эмбеддинг


In [66]:
# Определяем нейросетевую модель для коллаборативной фильтрации
class RecommenderNN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64):
        super(RecommenderNN, self).__init__()
        # Эмбеддинги пользователей и аниме
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        # Полносвязные слои для предсказания рейтинга
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_dim * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, user, item):
        # Получаем эмбеддинги пользователя и аниме
        user_emb = self.user_embedding(user)
        item_emb = self.item_embedding(item)

        # Объединяем эмбеддинги
        x = torch.cat([user_emb, item_emb], dim=1)

        # Пропускаем через полносвязные слои
        return self.fc_layers(x).squeeze()
    
# Определяем количество пользователей и аниме
num_users = len(user_id_map)
num_items = len(anime_id_map)

# Создаём датасеты и загрузчики данных
dataset = RatingsDataset(ratings_df)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


# Инициализация модели
model = RecommenderNN(num_users, num_items).to(device)

# Определяем функцию потерь (MSE) и оптимизатор (Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

epochs = 10

# Обучение модели
for epoch in range(epochs):
    model.train()
    total_loss = 0
    all_predictions = []
    all_ratings = []
    for users, items, ratings in train_loader:
        users, items, ratings = users.to(device), items.to(device), ratings.to(device)
        optimizer.zero_grad()
        predictions = model(users, items)
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        all_predictions.extend(predictions.cpu().detach().numpy())
        all_ratings.extend(ratings.cpu().detach().numpy())

    # Средняя ошибка предсказания на тренировочной выборке
    rmse = math.sqrt(mean_squared_error(all_ratings, all_predictions))
    mae = mean_absolute_error(all_ratings, all_predictions)

    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}, Train RMSE: {rmse:.4f}, Train MAE: {mae:.4f}')

    
# Оценка модели на тестовом наборе
model.eval()
test_predictions = []
test_ratings = []
with torch.no_grad():
    for users, items, ratings in test_loader:
        users, items, ratings = users.to(device), items.to(device), ratings.to(device)
        predictions = model(users, items)
        test_predictions.extend(predictions.cpu().numpy())
        test_ratings.extend(ratings.cpu().numpy())

# Средняя ошибка на тестовом наборе
test_rmse = math.sqrt(mean_squared_error(test_ratings, test_predictions))
test_mae = mean_absolute_error(test_ratings, test_predictions)

print(f'\nTest RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}')

idx_to_anime = {v: k for k, v in anime_id_map.items()}

# Выбираем случайных пользователей (оригинальные user_id)
random_users = np.random.choice(ratings_df['user_id'].unique(), size=5)

print("\nПерсонализированные рекомендации (топ-5 аниме) для случайных пользователей:")
for user_id in random_users:
    user_idx = user_id_map[user_id]
    
    # Предсказания для всех аниме для выбранного пользователя
    user_tensor = torch.tensor([user_idx] * num_items, dtype=torch.long).to(device)
    item_tensor = torch.tensor(range(num_items), dtype=torch.long).to(device)

    with torch.no_grad():
        predictions = model(user_tensor, item_tensor).cpu().numpy()

    # Выбираем топ-5 рекомендованных аниме (idx)
    top_indices = predictions.argsort()[-5:][::-1]
    top_anime_ids = [idx_to_anime[idx] for idx in top_indices]
    
    # Получаем названия
    top_titles = anime_df[anime_df['anime_id'].isin(top_anime_ids)]['title'].tolist()

    print(f"Пользователь {user_id}: Рекомендуемые аниме: {top_titles}")

Epoch 1, Loss: 1.534317907275505, Train RMSE: 1.2387, Train MAE: 0.9377
Epoch 2, Loss: 1.322184602504597, Train RMSE: 1.1499, Train MAE: 0.8671
Epoch 3, Loss: 1.2624238546147988, Train RMSE: 1.1236, Train MAE: 0.8459
Epoch 4, Loss: 1.2235452300890997, Train RMSE: 1.1061, Train MAE: 0.8320
Epoch 5, Loss: 1.187540085458911, Train RMSE: 1.0897, Train MAE: 0.8189
Epoch 6, Loss: 1.1504311103300022, Train RMSE: 1.0726, Train MAE: 0.8055
Epoch 7, Loss: 1.1133915421292284, Train RMSE: 1.0552, Train MAE: 0.7917
Epoch 8, Loss: 1.0796393847186074, Train RMSE: 1.0391, Train MAE: 0.7787
Epoch 9, Loss: 1.0522195427534786, Train RMSE: 1.0258, Train MAE: 0.7688
Epoch 10, Loss: 1.0256393670765422, Train RMSE: 1.0127, Train MAE: 0.7586

Test RMSE: 1.1133, Test MAE: 0.8409

Персонализированные рекомендации (топ-5 аниме) для случайных пользователей:
Пользователь 18309: Рекомендуемые аниме: ['Clannad', 'Clannad: After Story', 'Suzumiya Haruhi no Shoushitsu', 'Steins;Gate', 'Gintama・ゑｽｰ']
Пользователь 34459

Видим улучшение метрик как на эпохах так и на тестовой выборке. Увеличение количества эпох даст лишь переобучение